# 빅프9조

 필수 라이브러리 import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from selenium import webdriver as wb
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
import math

C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


상계1파출소

In [2]:
police = np.array([37.679699, 127.055066], dtype=np.float64)

상수 선언

In [3]:
NS_DIFF_1M = 0.000007
EW_DIFF_1M = 0.000009
PATROL_HR = 900
GRID_DIFF = 90
data_path = "data/"
encoding = "cp949"

# 기본 격자점 생성

원형으로 필터링된 격자점 반환

In [4]:
def filtering(police, grid):
    ft = (grid - police).copy()
    ft[:,0] /= NS_DIFF_1M
    ft[:,1] /= EW_DIFF_1M
    ft = np.sqrt( np.sum( np.square(ft), axis=1 ))
    grid = grid[ft <= PATROL_HR + 10, :]
    return grid

def grids(police):
    hcount = PATROL_HR//GRID_DIFF
    ew = np.arange(-hcount, hcount + 1, dtype=np.float64)
    ns = ew.reshape(-1,1).copy()
    ew *= EW_DIFF_1M * GRID_DIFF
    ns *= NS_DIFF_1M * GRID_DIFF
    
    result = np.full((2*hcount+1, 2*hcount+1, 2), police, dtype=np.float64)
    result[:,:,0] += ns
    result[:,:,1] += ew
    
    return filtering(police, result.reshape(-1,2))

격자점그리기

In [5]:
def draw_marker(m, grid, color):
    color = "#" + color
    for i in grid:
        folium.CircleMarker(location=i, radius=5, color=color).add_to(m)

지도객체 생성, 경찰서 특별 표식

In [6]:
m = folium.Map(location = police, zoom_start=60)
folium.Marker(location=police, 
              popup='지구대',
              icon=folium.Icon(color='lightblue', icon='star')
).add_to(m)

기본 격자점 생성

In [7]:
police_grid = pd.DataFrame(grids(police), columns=["위도", "경도"])

기본 격자점 그리기

In [8]:
draw_marker(m, police_grid.values, color='ffffff')
m

# 샘플링

뽑을 개수 (5개)

In [9]:
SAMPLING = 5

격자점 범위 내에서 무작위로 5개 점 추가

In [10]:
def sampled_grids(grid):
    interval_size = SAMPLING+1
    result = np.zeros((grid.shape[0]*interval_size, 2), dtype=np.float64)
    for i in range(0, result.shape[0], interval_size):
        ran = np.zeros((interval_size, 2), dtype=np.float64)
        ran[1:] = np.random.uniform(-1, 1, (interval_size-1, 2))
        ran[:, 0] *= (GRID_DIFF/2)*NS_DIFF_1M
        ran[:, 1] *= (GRID_DIFF/2)*EW_DIFF_1M
        ran += grid[i//interval_size]
        result[i:i+interval_size] += ran
    return result

추가한 데이터 csv내보내기

In [11]:
'''
police_sampled = sampled_grids(police_grid)
police_sampled

police_sampled = pd.DataFrame(police_sampled, columns = ["위도", "경도"])
police_sampled.to_csv(data_path + "상계1위경도샘플.csv", index=False, encoding=encoding)
'''

'\npolice_sampled = sampled_grids(police_grid)\npolice_sampled\n\npolice_sampled = pd.DataFrame(police_sampled, columns = ["위도", "경도"])\npolice_sampled.to_csv(data_path + "상계1위경도샘플.csv", index=False, encoding=encoding)\n'

https://www.geoservice.co.kr/

내보낸 csv 파일 이용, 위 링크에서 역지오코딩, 주소얻기

In [12]:
def addr_data(filename):
    police_addr_data=pd.read_csv(filename, encoding=encoding)
    police_addr_data["이름"] = None
    police_addr_data["확률"] = None
    police_addr_data["index"]=[i//(SAMPLING+1) for i in range(police_addr_data.shape[0])]
    return police_addr_data

police_addr_data = addr_data(data_path+"상계1샘플주소.csv")
police_addr_data

,위도,경도,지번주소,도로명주소,이름,확률,index
0,37.673399,127.054256,서울특별시 노원구 상계동 1044,서울특별시 노원구 동일로231길 24,None,None,0
1,37.673447,127.054045,서울특별시 노원구 상계동 1044,서울특별시 노원구 동일로231길 24,None,None,0
2,37.673578,127.054242,서울특별시 노원구 상계동 1044,서울특별시 노원구 동일로231길 24,None,None,0
3,37.673156,127.054325,서울특별시 노원구 상계동 1044,서울특별시 노원구 동일로231길 24,None,None,0
4,37.673209,127.054497,서울특별시 노원구 상계동 1044-2,서울특별시 노원구 동일로 1625,None,None,0
...,...,...,...,...,...,...,...
1945,37.686257,127.055793,서울특별시 노원구 상계동 산 12-1,서울특별시 노원구 동일로 1772,None,None,324
1946,37.685699,127.055507,서울특별시 노원구 상계동 1202-1,서울특별시 노원구 누원로 18,None,None,324
1947,37.686189,127.055881,서울특별시 노원구 상계동 산 12-1,서울특별시 노원구 동일로 1772,None,None,324
1948,37.685953,127.055901,서울특별시 노원구 상계동 산 12-1,서울특별시 노원구 동일로 1772,None,None,324


# 유형 크롤링

지연 시간(1초), 웹 지도 url(카카오맵) 설정

In [13]:
DELAY_TIME = 0.5
map_url = "https://map.kakao.com/"

브라우저 드라이버 반환(no 전달시 창 안뜸)

In [14]:
def openweb(mode="window"):
    if mode == "no":
        options = wb.ChromeOptions()
        options.add_argument('headless')
        driver = wb.Chrome(options=options)
    else:
        driver = wb.Chrome()
    driver.get(map_url)
    time.sleep(DELAY_TIME)
    return driver

검색만 함

In [15]:
def search(browser, address):
    search_box = browser.find_element(By.ID, "search.keyword.query")
    while search_box.get_attribute("value"):
        search_box.send_keys(Keys.CONTROL, 'a')
        search_box.send_keys(Keys.BACKSPACE)
    search_box.send_keys(address)
    search_box.send_keys(Keys.RETURN)
    time.sleep(DELAY_TIME)

건물 이름 찾아 반환 안뜨면 우편번호 찾기 안뜨면 nozip 산은 산

In [16]:
def find_name(browser, addr, road_addr):
    if "산" in addr:
        return "산"
    if "지하" in road_addr:
        return "지하철"
    search(browser, addr)
    try:
        buildname = browser.find_element(By.CLASS_NAME, "building")
    except NoSuchElementException:
        try:
            postage = browser.find_element(By.CLASS_NAME, "zip")
        except NoSuchElementException:
            return "nozip"
        else:
            return postage.text
    else:
        return buildname.text

주소 전부 검색 후 csv로 내보내기

In [17]:
'''
browser = openweb("no")
for i in range(len(police_addr_data.index)):
    police_addr_data.iloc[i, 4] = find_name(browser, police_addr_data.iloc[i, 2], police_addr_data.iloc[i, 3])
police_addr_data.to_csv(data_path+"유형.csv", index=False, encoding=encoding)
'''

'\nbrowser = openweb("no")\nfor i in range(len(police_addr_data.index)):\n    police_addr_data.iloc[i, 4] = find_name(browser, police_addr_data.iloc[i, 2], police_addr_data.iloc[i, 3])\npolice_addr_data.to_csv(data_path+"유형.csv", index=False, encoding=encoding)\n'

In [18]:
police_addr_data = pd.read_csv(data_path + "유형.csv", encoding=encoding)
set(police_addr_data["이름"])

{'(우) 01319',
 '(우) 01320',
 '(우) 01321',
 '(우) 01323',
 '(우) 01604',
 '(우) 01605',
 '(우) 01606',
 '(우) 01608',
 '(우) 01609',
 '(우) 01610',
 '(우) 01611',
 '(우) 01612',
 '(우) 01621',
 '(우) 01622',
 '(우) 01623',
 '(우) 01625',
 '(우) 01626',
 '(우) 01627',
 '(우) 01628',
 '(우) 01629',
 '(우) 01630',
 '(주)씨에스윈',
 'K마트',
 'nozip',
 '가로판매대',
 '경서아트빌',
 '구두수선대',
 '구립상일경로당',
 '극동아파트',
 '기독교대한성결교회영광교회',
 '기아자동차서비스도봉사업소',
 '남광하이빌',
 '노원교회',
 '노일중학교',
 '노일초등학교',
 '누원고등학교',
 '대건주택',
 '대망드림힐아파트',
 '대명하우스',
 '대원파크빌',
 '대은빌딩',
 '대한빌딩',
 '도봉기적의도서관',
 '도봉동 실내스포츠센터',
 '도봉파크빌2단지',
 '도봉한신아파트',
 '도선어린이집',
 '도우로펌타워',
 '도현빌딩',
 '동아에코빌아파트',
 '동창빌딩',
 '동해회',
 '두산아파트',
 '래미안도봉아파트',
 '롯데리아',
 '르비앙',
 '모던하우스',
 '미소홈타운',
 '바이크종합매장',
 '백승빌라',
 '보미네순대국',
 '북경짜장',
 '북부종합사회복지관',
 '북부현대아파트',
 '비콘드림힐아파트',
 '사랑요양병원',
 '산',
 '삼덕빌딩',
 '삼성디지털센터',
 '삼성프라임',
 '삼환도봉아파트',
 '상계1동우체국',
 '상계1동주민센터',
 '상계1파출소',
 '상계빌딩',
 '상록수2단지아파트',
 '상일교회',
 '새온교회',
 '서광아파트',
 '서울북부지방법원',
 '서울수락초등학교',
 '서울해장국',
 '서원아파트',
 '성균관대학교종합운동장',
 '성원빌딩',
 '세주

범죄장소데이터 확률로 가공

In [19]:
'''
crime_loc = pd.read_csv(data_path+"범죄장소.csv", encoding='cp949')
crime_loc = crime_loc.groupby("장소")
crime_loc = crime_loc.sum()
crime_loc /= crime_loc.sum()
crime_loc.to_csv(data_path + "범죄장소확률.csv", encoding=encoding)
'''

'\ncrime_loc = pd.read_csv(data_path+"범죄장소.csv", encoding=\'cp949\')\ncrime_loc = crime_loc.groupby("장소")\ncrime_loc = crime_loc.sum()\ncrime_loc /= crime_loc.sum()\ncrime_loc.to_csv(data_path + "범죄장소확률.csv", encoding=encoding)\n'

비슷한 목록 합치기

In [20]:
'''
per = pd.DataFrame()
per["공업"] = crime_per[["공사장광산", "공장", "창고"]].sum(axis=1)
per["교통"] = crime_per[["기타교통수단내", "지하철", "역,대합실"]].sum(axis=1)
per["사무"] = crime_per[["금융기관", "사무실"]].sum(axis=1)
per["숙박"] = crime_per[["숙박업소목욕탕"]].sum(axis=1)
per["상업"] = crime_per[["시장,노점", "상점", "흥행장"]].sum(axis=1)
per["아파트"] = crime_per[["아파트,연립다세대"]].sum(axis=1)
per["주택"] = crime_per[["단독주택"]].sum(axis=1)
per["유흥"] = crime_per[["유흥접객업소"]].sum(axis=1)
per["공원"] = crime_per[["유원지", "공지"]].sum(axis=1)
per["의료"] = crime_per[["의료기관"]].sum(axis=1)
per["종교"] = crime_per[["종교기관"]].sum(axis=1)
per["학교"] = crime_per[["학교"]].sum(axis=1)
per["기타"] = crime_per[["구금장소", "고속도로", "기타", "부대"]].sum(axis=1)
per["야외"] = crime_per[["노상", "산야"]].sum(axis=1)
per
'''

'\nper = pd.DataFrame()\nper["공업"] = crime_per[["공사장광산", "공장", "창고"]].sum(axis=1)\nper["교통"] = crime_per[["기타교통수단내", "지하철", "역,대합실"]].sum(axis=1)\nper["사무"] = crime_per[["금융기관", "사무실"]].sum(axis=1)\nper["숙박"] = crime_per[["숙박업소목욕탕"]].sum(axis=1)\nper["상업"] = crime_per[["시장,노점", "상점", "흥행장"]].sum(axis=1)\nper["아파트"] = crime_per[["아파트,연립다세대"]].sum(axis=1)\nper["주택"] = crime_per[["단독주택"]].sum(axis=1)\nper["유흥"] = crime_per[["유흥접객업소"]].sum(axis=1)\nper["공원"] = crime_per[["유원지", "공지"]].sum(axis=1)\nper["의료"] = crime_per[["의료기관"]].sum(axis=1)\nper["종교"] = crime_per[["종교기관"]].sum(axis=1)\nper["학교"] = crime_per[["학교"]].sum(axis=1)\nper["기타"] = crime_per[["구금장소", "고속도로", "기타", "부대"]].sum(axis=1)\nper["야외"] = crime_per[["노상", "산야"]].sum(axis=1)\nper\n'

장소 분류에 따른 범죄확률

In [21]:
crime_per = pd.read_csv(data_path+"범죄장소확률.csv", encoding=encoding)
crime_per

,공업,교통,사무,숙박,상업,아파트,주택,유흥,공원,의료,종교,학교,기타,야외
0,0.00583,0.043463,0.025199,0.010841,0.123032,0.108198,0.087457,0.065698,0.010653,0.007179,0.001892,0.004735,0.180887,0.324937


분류 결과를 바탕으로 이름이 어떤유형인지 자료화

자료에 맞춰 확률 계산후 대입

In [22]:
def namespace(filename):
    space = dict()
    with open(filename, 'r', encoding='UTF8') as file:
        for i in file:
            temp=i.split(" - ")
            space[temp[0]] = temp[1].strip()
    return space

def insert_percentage(filename, addr_data):
    space = namespace(filename)
    for i in space:
        addr_data.loc[addr_data["이름"]==i, "확률"] = crime_per.loc[0, space[i]]

insert_percentage(data_path + "이름유형분류.txt", police_addr_data)
        
police_addr_data

,위도,경도,지번주소,도로명주소,이름,확률,index
0,37.673399,127.054256,서울특별시 노원구 상계동 1044,서울특별시 노원구 동일로231길 24,노일중학교,0.004735,0
1,37.673447,127.054045,서울특별시 노원구 상계동 1044,서울특별시 노원구 동일로231길 24,노일중학교,0.004735,0
2,37.673578,127.054242,서울특별시 노원구 상계동 1044,서울특별시 노원구 동일로231길 24,노일중학교,0.004735,0
3,37.673156,127.054325,서울특별시 노원구 상계동 1044,서울특별시 노원구 동일로231길 24,노일중학교,0.004735,0
4,37.673209,127.054497,서울특별시 노원구 상계동 1044-2,서울특별시 노원구 동일로 1625,노일초등학교,0.004735,0
...,...,...,...,...,...,...,...
1945,37.686257,127.055793,서울특별시 노원구 상계동 산 12-1,서울특별시 노원구 동일로 1772,산,0.324937,324
1946,37.685699,127.055507,서울특별시 노원구 상계동 1202-1,서울특별시 노원구 누원로 18,nozip,0.324937,324
1947,37.686189,127.055881,서울특별시 노원구 상계동 산 12-1,서울특별시 노원구 동일로 1772,산,0.324937,324
1948,37.685953,127.055901,서울특별시 노원구 상계동 산 12-1,서울특별시 노원구 동일로 1772,산,0.324937,324


동일 지역에서 샘플된 데이터끼리 확률 합 구한다

In [23]:
def insert_score(grid, percentage_data):
    score = percentage_data[["index", "확률"]].groupby("index").sum()
    grid["점수"] = score

In [24]:
insert_score(police_grid, police_addr_data)
police_grid

,위도,경도,점수
0,37.673399,127.054256,0.028407
1,37.673399,127.055066,0.028407
2,37.673399,127.055876,1.070853
3,37.674029,127.051826,0.762221
4,37.674029,127.052636,0.999701
...,...,...,...
320,37.685369,127.057496,1.949623
321,37.685369,127.058306,1.949623
322,37.685999,127.054256,0.649189
323,37.685999,127.055066,1.082667


시각화를 위해 점수를 -255(초록)~255(빨강)으로 변환

변환후 rgb코드에 맞게 문자열 생성 안전->초록, 위험->빨강

In [25]:
def colorstr(num):
    b = num>=0
    num=hex(int(255-abs(num)))[2:]
    if len(num)==1:
        num="0"+num
    return "#ff"+num*2 if b else "#"+num+"ff"+num

In [26]:
def score_to_color(grid):
    result = grid["점수"].copy()
    mins = result.min()
    maxs = result.max()
    result = ((result - mins) / (maxs - mins) * 2 - 1) * 255
    result = result.apply(colorstr)
    return result

In [29]:
def draw_score(grid, color):
    for i in grid.index:
        folium.CircleMarker(location=grid.iloc[i,:2].values,
                            radius=20,
                            color=color[i],
                            fill=True).add_to(m)

장소에 따른 위험도 시각화

In [27]:
police_grid_color = score_to_color(police_grid)
police_grid_color

0      #00ff00
1      #00ff00
2      #ffe9e9
3      #c2ffc2
4      #fffcfc
        ...   
320    #ff0000
321    #ff0000
322    #a4ffa4
323    #ffe6e6
324    #ff0000
Name: 점수, Length: 325, dtype: object

In [31]:
draw_score(police_grid, police_grid_color)
m

해당 범위 안에 목표물이 몇개 있는가?

In [ ]:
#격자점반경에 대해
def location_count(grid, location):
    temp = (grid - location).copy()
    temp.iloc[:,0] /= NS_DIFF_1M
    temp.iloc[:,1] /= EW_DIFF_1M
    temp = np.sqrt( np.sum( np.square(temp), axis=1 ))
    return len(grid.loc[temp <= GRID_DIFF/2, :].index)

#순찰반경에 대해
def police_count(grid, police):
    temp = (grid - police).copy()
    temp.iloc[:,0] /= NS_DIFF_1M
    temp.iloc[:,1] /= EW_DIFF_1M
    temp = np.sqrt( np.sum( np.square(temp), axis=1 ))
    return len(grid.loc[temp <= PATROL_HR+GRID_DIFF/2, :].index)

### 안심시설물 데이터 이용

In [ ]:
'''
data = pd.read_csv("안전시설물.csv", encoding='cp949')
def dong_guard_data(data, dong):
    re=""
    for i in dong:
        re+= i+"|"
    data = data.loc[data["읍면동명"].str.contains(re[:-1]) , ["포인트 wkt", "읍면동명"]]
    def decompose(word):
        temp = word.split("(")
        return temp[-1][:-1]
    def NS(word):
        temp = word.split(" ")
        return temp[1]
    def EW(word):
        temp = word.split(" ")
        return temp[0]
    data["포인트 wkt"] = data["포인트 wkt"].apply(decompose)
    data["위도"] = data["포인트 wkt"].apply(NS)
    data["위도"] = data["위도"].astype(np.float64)
    data["경도"] = data["포인트 wkt"].apply(EW)
    data["경도"] = data["경도"].astype(np.float64)
    return data.drop(columns=["포인트 wkt", "읍면동명"]).reset_index(drop=True)

data = dong_guard_data(data, ["도봉동", "상계동"])
'''

In [34]:
ansim_data = pd.read_csv(data_path+"상계1안심시설물.csv", encoding=encoding)
ansim_data

,위도,경도
0,37.677522,127.044616
1,37.677482,127.044476
2,37.677482,127.044476
3,37.677289,127.044185
4,37.677170,127.044018
...,...,...
273,37.675134,127.053706
274,37.675134,127.053706
275,37.675047,127.054265
276,37.675130,127.054613


In [ ]:
#draw_marker(m, ansim_data.values, color='0000ff')
m

### cctv 데이터 이용

In [35]:
dobong_cctv=pd.read_csv(data_path+"dobong_cctv.csv", encoding=encoding)
nowon_cctv=pd.read_csv(data_path+"nowon_cctv.csv", encoding=encoding)

In [36]:
cctv = pd.concat([nowon_cctv, dobong_cctv])
cctv

,위도,경도
0,37.6578,127.0613
1,37.6575,127.0614
2,37.6591,127.0568
3,37.6590,127.0565
4,37.6683,127.0620
...,...,...
1593,37.6595,127.0450
1594,37.6545,127.0460
1595,37.6555,127.0420
1596,37.6530,127.0431


In [ ]:
#draw_marker(m, cctv.values, color='00ffff')
m

In [37]:
cctv.to_csv(data_path+"nowon_dobong_cctv.csv", encoding=encoding)

### 보안등 데이터 이용

보안등은 너무 많아서 순찰반경내의 것만 잘라 사용

In [38]:
def light_cut(police, light_data):
    NS_MAX=police[0]+(PATROL_HR+GRID_DIFF/2)*NS_DIFF_1M
    NS_MIN=police[0]-(PATROL_HR+GRID_DIFF/2)*NS_DIFF_1M
    EW_MAX=police[1]+(PATROL_HR+GRID_DIFF/2)*EW_DIFF_1M
    EW_MIN=police[1]-(PATROL_HR+GRID_DIFF/2)*EW_DIFF_1M
    return light_data.loc[(NS_MIN<=light_data["위도"])&(light_data["위도"]<=NS_MAX)&
                           (EW_MIN<=light_data["경도"])&(light_data["경도"]<=EW_MAX),:]

In [40]:
dobong_light = pd.read_csv(data_path+"dobong_light.csv", encoding=encoding)
nowon_light = pd.read_csv(data_path+"nowon_light.csv", encoding=encoding)
light_data = pd.concat([dobong_light, nowon_light])
light_data

,위도,경도
0,37.659426,127.036036
1,37.660503,127.040867
2,37.658717,127.038386
3,37.653005,127.035297
4,37.656666,127.037624
...,...,...
652486,37.623603,127.063553
652487,37.623819,127.063466
652488,37.623776,127.063446
652489,37.623573,127.063403


In [41]:
light_police = light_cut(police, light_data)
light_police

,위도,경도
2136,37.682286,127.048089
2143,37.673506,127.047651
2145,37.673607,127.047563
2150,37.673676,127.047794
2153,37.675071,127.046822
...,...,...
652323,37.677328,127.052505
652330,37.674857,127.059506
652333,37.676011,127.057590
652368,37.679208,127.052209


In [ ]:
#draw_marker(m, light_police.values, color='ffff00')
m